In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from model import Model

torch.set_printoptions(linewidth=1000000)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Load trained model
Load the model from a trained checkpoint

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
model = Model.load_from_checkpoint('flan-t5-base-batch4.ckpt')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', truncation_side='left')
model.eval()

Model(
  (model): PeftModelForSeq2SeqLM(
    (base_model): LoraModel(
      (model): T5ForConditionalGeneration(
        (shared): Embedding(32128, 768)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 768)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): Linear(
                      in_features=768, out_features=768, bias=False
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=768, out_features=16, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=16, out_features=768, bias=False)
                      )
                      (lora_embeddi

# Configure dialog with model
Creating an action to easily "communicate" with the model

In [4]:
context = {}

def dialog(topic, prompt):
    with torch.no_grad():
        current_context = ''

        if topic in context:
            context[topic].append(prompt)
        else:
            context[topic] = []
            context[topic].append(prompt)

        current_context = ' '.join(context[topic])
        tokenized_input = tokenizer(current_context, return_tensors="pt").to(device)

        generated = model.model.generate(**tokenized_input, do_sample=True, top_p=0.9, max_new_tokens=200, no_repeat_ngram_size =5)
        print(generated.dtype)
        generated_text = tokenizer.decode(generated[0], skip_special_token=True)
        context[topic].append(generated_text)

        return generated_text
    

In [5]:
print(dialog('eval', 'What is the white house?'))

torch.int64
<pad> White House is the White House building in Washington, D.C., located on Washington’s North Side. It was built to house President Bush and is currently the White House’s first building. The White House is also known for its mission statement: “To build a more peaceful, prosperous and prosperous country, the white House is an invaluable tool for all Americans, including Americans in the United States and other countries.” The White House is the largest house in the United States. The White House was founded in 1867 to house President Bush’s administration, which includes the White House’ spokesman, Paul Ryan, and several members of his Cabinet. The White House also has a number of other offices, including the White House’ Senate, the White House’ White House’s Office of Budget and Budget, and the White House’ congressional office. The White House has several rooms, including the Whitehouse’s lobbyist, the White House Press Association and the White House


In [6]:
print(dialog('eval', 'Who lives there?'))

torch.int64
<pad> The White House is located in Washington, D.C., where President Bush was born. While many Americans currently live in Washington, D, they enjoy the privacy of the White House, and have access to the iconic building's unique spaces and facilities. One popular place to live in the White House is Washington Square Park, which is a popular spot for shopping and shopping. The White House has a lot of spaces for socializing, including an outdoor patio, and is located near the Washington Square Park Zoo. Other popular spots for socializing are the White House's new restaurants and hotels. One area where the White House is located, known as The Park, is the White House' Senate office. Additionally, there are several other spaces to live in, including a lobbyist's office, a conference room, and a small office. The White House is also home to the White House Press Association, which hosts a weekly news show on the White House.


In [7]:
print(dialog('python', 'What is python for?'))

torch.int64
<pad> Python is a code-driven platform used by the Python community to create and maintain code. Python is a distributed application that uses the Python syntax and compiled code to build and maintain code in Python. <unk>python <unk>http://python.com/<unk>pythonic <unk>pyton <unk>/python<unk> <unk>pytheon <unk>pypython_<unk>pythont_<unk>pythetons; <unk>pyhetons.com<unk>pythettons.com<unk>Pythetons;<unk>pythlets.com<unk>Pmythetons<unk>pythtons;<unk>psythetons;psythetes.com<unk>psythtons.com<unk>. <unk>pxythtons.pythetons.com<unk>t<unk>pythen.com<unk>pzthen.com;


In [8]:
print(dialog('translation', 'Translate from german to english: Hallo mein Name ist flan-t5'))

torch.int64
<pad> Hallo mein Name ist flan-t5</s>


In [9]:
print(dialog('translation', 'Translate it to french'))

torch.int64
<pad> Hallo mein Name ist Flan-T5.</s>


In [10]:
print(dialog('word', 'How can I use microsoft word?'))

torch.int64
<pad> You can use a copy of the document to save the document, then to access the document by using the user's name and email address in the document. Here is a brief description of how to use Microsoft Word: Microsoft Word: <unk> <unk> Microsoft Word: Using Microsoft Word can be a great way to learn how to use the app and the app's functions, including: - <unk> Microsoft Word Search: <unk> Microsoft Word allows users to search for a specific app, such as a website or email, in a web browser, or through other online search engines. - <unk> <unk> <unk> Word Search: Using Microsoft Google Translate: Using Microsoft <unk> <unk> If you want to use the app or app, you can use the <unk> <unk> toolbar<unk> on your web browser or app, or download it from a website. - Scripting: Using Microsoft Chrome for editing and saving


In [11]:
print(dialog('word', 'Do I have to buy it to use it?'))

torch.int64
<pad> Yes! You can download Microsoft Word from https://www.microsoft.com/ and install it on your computer. Once you're set up, you can take the.embc file to your computer and set it up on your computer. To use the app, you can download it from https://www1.microsoft.com/ to install it on your desktop computer. The installation process can take anywhere from two hours to three hours, depending on your computer. You can download Microsoft PowerPoint from https://www2.microsoft.com/ as a PDF file, or you can download it directly from https://www4.microsoft.com/ at your computer's website. Here is the complete installation process: If you don't see the installation, you can use Microsoft PowerPoint. You can download a copy of the Microsoft PowerPoint file on your computer, then use Microsoft PowerPoint. The file can be downloaded from https://www3.microsoft.com/ after you install the app on


In [12]:
print(dialog('word', 'Where can I buy it?'))

torch.int64
<pad> You can download Microsoft Word from https://www.microsoft.com/ and install it on your computer. Once you're set up, you can take the.embc file to your computer and set it up on your computer. To use the app, you can download it from https://www1.microsoft.com/ to install it on your desktop computer. The installation process can take anywhere from two hours to three hours, depending on your computer. You can download Microsoft PowerPoint from https://www2.microsoft.com/ as a PDF file, or you can download it directly from https://www4.microsoft.com/ at your computer's website. Here is the complete installation process: If you don't see the installation, you can use Microsoft PowerPoint. You can download a copy of the Microsoft PowerPoint file on your computer, then use Microsoft PowerPoint. The file can be downloaded from https://www3.microsoft.com/ after you install it on your PC. You
